# Intrusion Detection and Prevention System

Dataset: CIC-IDS2017

https://www.unb.ca/cic/datasets/ids-2017.html

https://www.kaggle.com/datasets/chethuhn/network-intrusion-dataset/data

CIC-IDS2017 dataset contains benign and the most up-to-date common attacks, which resembles the true real-world data.

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes=True)
import glob
import os

In [4]:
# 1. Get list of all CSV files from the CIC-IDS2017 dataset
path = "./CIC-IDS2017/"
all_files = glob.glob(path + "*.csv")

li = []

for filename in all_files:
    print(f"Loading {os.path.basename(filename)}...")

    # Read the CSV
    # 'encoding' is often needed for this specific dataset to avoid errors
    df_temp = pd.read_csv(filename, encoding='cp1252', low_memory=False)

    # CRITICAL FIX: Strip whitespace from column names immediately
    # This prevents " Label" and "Label" from being treated as different columns
    df_temp.columns = df_temp.columns.str.strip()

    # OPTIONAL: Memory Saver
    # If the file is 'Monday', skip to avoid major class imbalance
    if 'Monday' in filename:
        continue

    li.append(df_temp)

# 2. Merge all into one DataFrame
print("Concatenating files...")
df = pd.concat(li, axis=0, ignore_index=True)

print(f"Final Dataset Shape: {df.shape}")

Loading Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv...
Loading Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv...
Loading Friday-WorkingHours-Morning.pcap_ISCX.csv...
Loading Monday-WorkingHours.pcap_ISCX.csv...
Loading Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv...
Loading Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv...
Loading Tuesday-WorkingHours.pcap_ISCX.csv...
Loading Wednesday-workingHours.pcap_ISCX.csv...
Concatenating files...
Final Dataset Shape: (2300825, 79)


The data capturing period started at 9 a.m., Monday, July 3, 2017 and ended at 5 p.m. on Friday July 7, 2017, for a total of 5 days. Monday is the normal day and only includes the benign traffic. The implemented attacks include Brute Force FTP, Brute Force SSH, DoS, Heartbleed, Web Attack, Infiltration, Botnet and DDoS. They have been executed both morning and afternoon on Tuesday, Wednesday, Thursday and Friday.

# Phase 1: Data Handling

## Exploratory Data Analysis

In [5]:
df.dtypes

Destination Port                 int64
Flow Duration                    int64
Total Fwd Packets                int64
Total Backward Packets           int64
Total Length of Fwd Packets      int64
                                ...   
Idle Mean                      float64
Idle Std                       float64
Idle Max                         int64
Idle Min                         int64
Label                           object
Length: 79, dtype: object

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2300825 entries, 0 to 2300824
Data columns (total 79 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Destination Port             int64  
 1   Flow Duration                int64  
 2   Total Fwd Packets            int64  
 3   Total Backward Packets       int64  
 4   Total Length of Fwd Packets  int64  
 5   Total Length of Bwd Packets  int64  
 6   Fwd Packet Length Max        int64  
 7   Fwd Packet Length Min        int64  
 8   Fwd Packet Length Mean       float64
 9   Fwd Packet Length Std        float64
 10  Bwd Packet Length Max        int64  
 11  Bwd Packet Length Min        int64  
 12  Bwd Packet Length Mean       float64
 13  Bwd Packet Length Std        float64
 14  Flow Bytes/s                 float64
 15  Flow Packets/s               float64
 16  Flow IAT Mean                float64
 17  Flow IAT Std                 float64
 18  Flow IAT Max                 int64  
 19  

All but the 'Label' column appear to already be in numerical format.

In [7]:
df.describe()

d:\BSAI\Projects\Intrusion Detection & Resolution\venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
d:\BSAI\Projects\Intrusion Detection & Resolution\venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,...,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06,2.300825e+06
mean,7.478905e+03,1.579823e+07,9.124129e+00,1.013505e+01,5.531908e+02,1.576287e+04,2.114469e+02,1.835353e+01,5.991961e+01,7.154907e+01,...,4.958900e+00,-2.540647e+03,8.457227e+04,4.065386e+04,1.549771e+05,6.163359e+04,9.433560e+06,5.732621e+05,9.864630e+06,8.990073e+06
std,1.743643e+04,3.460566e+07,7.127588e+02,9.522747e+02,1.067420e+04,2.156969e+06,7.657296e+02,6.461895e+01,2.015989e+02,3.038008e+02,...,5.755252e+02,1.173879e+06,6.618916e+05,3.925080e+05,1.025175e+06,5.935032e+05,2.532916e+07,4.995685e+06,2.611131e+07,2.506222e+07
min,0.000000e+00,-1.300000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-5.368707e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.300000e+01,1.470000e+02,1.000000e+00,1.000000e+00,6.000000e+00,2.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.000000e+01,3.132200e+04,2.000000e+00,2.000000e+00,6.000000e+01,1.200000e+02,3.600000e+01,2.000000e+00,3.300000e+01,0.000000e+00,...,1.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.430000e+02,4.582168e+06,5.000000e+00,4.000000e+00,1.870000e+02,5.680000e+02,8.100000e+01,3.500000e+01,4.928571e+01,2.327660e+01,...,2.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553300e+04,1.200000e+08,2.079640e+05,2.846020e+05,1.290000e+07,6.270000e+08,2.482000e+04,2.325000e+03,5.940857e+03,7.049469e+03,...,1.986360e+05,1.380000e+02,1.100000e+08,7.420000e+07,1.100000e+08,1.100000e+08,1.200000e+08,7.690000e+07,1.200000e+08,1.200000e+08


In [8]:
null_counts = df.isnull().sum()
print(null_counts if (null_counts > 0).any() else "No missing values")

Destination Port               0
Flow Duration                  0
Total Fwd Packets              0
Total Backward Packets         0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
Idle Std                       0
Idle Max                       0
Idle Min                       0
Label                          0
Length: 79, dtype: int64


In [9]:
missing_value_columns = null_counts[null_counts > 0]
if not missing_value_columns.empty:
    print("Columns with missing values:")
    print(missing_value_columns)
else:
    print("No columns with missing values.")

Columns with missing values:
Flow Bytes/s    1294
dtype: int64


In [10]:
df['Flow Bytes/s'].isnull().sum()

np.int64(1294)

In [11]:
df['Flow Bytes/s'].head()

0    4.000000e+06
1    1.100917e+05
2    2.307692e+05
3    3.529412e+05
4    4.000000e+06
Name: Flow Bytes/s, dtype: float64

In [12]:
df['Flow Bytes/s'].describe()

d:\BSAI\Projects\Intrusion Detection & Resolution\venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


count    2.299531e+06
mean              inf
std               NaN
min     -2.610000e+08
25%      1.182046e+02
50%      4.292120e+03
75%      1.600000e+05
max               inf
Name: Flow Bytes/s, dtype: float64

The data appears to have infinite and NaN values that need to be handled

In [13]:
df['Label'].value_counts()

Label
BENIGN                          1743179
DoS Hulk                         231073
PortScan                         158930
DDoS                             128027
DoS GoldenEye                     10293
FTP-Patator                        7938
SSH-Patator                        5897
DoS slowloris                      5796
DoS Slowhttptest                   5499
Bot                                1966
Web Attack ï¿½ Brute Force         1507
Web Attack ï¿½ XSS                  652
Infiltration                         36
Web Attack ï¿½ Sql Injection         21
Heartbleed                           11
Name: count, dtype: int64

Major class imbalance with some classes having very few instances that the neural network will fail to learn properly.

## Preprocessing

In [14]:
# 1. Fix the weird characters first
df['Label'] = df['Label'].str.replace('ï¿½', '-', regex=False)
df['Label'] = df['Label'].str.strip()

# 2. Define the consolidation dictionary
# Mapping specific attacks to broader categories
# This will address the major class imbalance
# Many attacks will require the same response so they are combined to form common categories
attack_grouping = {
    'BENIGN': 'Normal',

    # Grouping all DoS attempts
    'DoS Hulk': 'DoS',
    'DoS GoldenEye': 'DoS',
    'DoS slowloris': 'DoS',
    'DoS Slowhttptest': 'DoS',

    # DDoS is distinct enough (distributed) to keep separate
    'DDoS': 'DDoS',

    # Port Scans are distinct pre-attack reconnaissance
    'PortScan': 'PortScan',

    # Grouping Web Attacks
    'Web Attack - Brute Force': 'WebAttack',
    'Web Attack - XSS': 'WebAttack',
    'Web Attack - Sql Injection': 'WebAttack',

    # Grouping Brute Force / Credential Stuffing
    'FTP-Patator': 'BruteForce',
    'SSH-Patator': 'BruteForce',

    # Keeping Botnet separate
    'Bot': 'Botnet'
}

# 3. Apply the mapping
df['Label_Category'] = df['Label'].map(attack_grouping)

# 4. Handle the leftovers (Rare classes like Heartbleed/Infiltration)
# Any label not in the dictionary will become NaN, so we drop them
df = df.dropna(subset=['Label_Category'])

# Check the new distribution
print("New Consolidated Distribution:")
print(df['Label_Category'].value_counts())

New Consolidated Distribution:
Label_Category
Normal        1743179
DoS            252661
PortScan       158930
DDoS           128027
BruteForce      13835
WebAttack        2180
Botnet           1966
Name: count, dtype: int64


In [15]:
# Check for infinite or null values in the feature columns
print("Contains Infinity:", np.isinf(df.select_dtypes(include=np.number)).values.any())
print("Contains NaN:", df.isnull().values.any())

Contains Infinity: True
Contains NaN: True


Infinite values are a result of division by zero in the 'Flow Bytes/s' column.

In [16]:
from sklearn.preprocessing import LabelEncoder

# Handle Infinity and NaN: Replace Infinity with NaN first, then drop all NaNs
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

print(f"Shape after cleaning: {df.shape}")

# Label Encoding
le = LabelEncoder()
df['Label_Encoded'] = le.fit_transform(df['Label_Category'])

label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label Mapping:", label_mapping)

# Drop the original string columns to save memory
df = df.drop(columns=['Label', 'Label_Category'])

Shape after cleaning: (2298348, 80)
Label Mapping: {'Botnet': np.int64(0), 'BruteForce': np.int64(1), 'DDoS': np.int64(2), 'DoS': np.int64(3), 'Normal': np.int64(4), 'PortScan': np.int64(5), 'WebAttack': np.int64(6)}


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

y = df['Label_Encoded']
X = df.drop(columns=['Label_Encoded'])

# stratify=y ensures we keep the same ratio of attacks in both sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training shape: {X_train_scaled.shape}")

Training shape: (1838678, 78)


In [18]:
X_train_scaled[:1]

array([[8.08752842e-04, 2.47610311e-01, 4.84390516e-06, 7.24448695e-06,
        1.05426357e-05, 7.59170654e-07, 3.10233683e-03, 2.53763441e-02,
        1.14461598e-02, 1.80551500e-03, 1.62314388e-02, 5.49033149e-02,
        4.10309456e-02, 1.36336181e-02, 1.11969121e-01, 3.33333356e-01,
        8.25368411e-02, 2.00471698e-01, 2.46666748e-01, 2.55174970e-04,
        2.47500000e-01, 2.47500000e-01, 0.00000000e+00, 2.47500000e-01,
        2.47500006e-01, 2.46666667e-01, 2.46666667e-01, 0.00000000e+00,
        2.46666667e-01, 2.46666667e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 9.99866829e-01, 9.94884123e-01,
        2.24366970e-08, 3.36550455e-08, 4.07458564e-02, 1.27719581e-02,
        4.02140411e-02, 2.32993831e-02, 5.42553571e-04, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.41025641e-03,
        4.30864726e-02, 1.14461598e-02, 4.10309456e-02, 9.998668

# Phase 2 Intrusion Detection Module

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import accuracy_score

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Convert Numpy arrays to PyTorch Tensors
# Note: PyTorch expects Float32 by default
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to(device)

# Create DataLoaders (Mini-batch handling)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

# Define the Neural Network
class AnalystModel(nn.Module):
    def __init__(self, input_shape, num_classes):
        super(AnalystModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_shape, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(64, 32),
            nn.ReLU(),
            
            # Output layer
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        return self.network(x)

# Initialize
input_shape = X_train_scaled.shape[1]
num_classes = len(np.unique(y_train))
model = AnalystModel(input_shape, num_classes).to(device)

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 20
print("Starting Training...")

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()           # Clear gradients
        outputs = model(inputs)         # Forward pass
        loss = criterion(outputs, labels) # Calc loss
        loss.backward()                 # Backward pass
        optimizer.step()                # Update weights
        running_loss += loss.item()
        
    print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss/len(train_loader):.4f}")

# Evaluation
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    acc = accuracy_score(y_test_tensor.cpu(), predicted.cpu())
    
print(f"Final Test Accuracy: {acc * 100:.2f}%")

Using device: cpu
Starting Training...
Epoch 1/20 - Loss: 0.2074
Epoch 2/20 - Loss: 0.0875
Epoch 3/20 - Loss: 0.0687
Epoch 4/20 - Loss: 0.0616
Epoch 5/20 - Loss: 0.0576
Epoch 6/20 - Loss: 0.0546
Epoch 7/20 - Loss: 0.0522
Epoch 8/20 - Loss: 0.0509
Epoch 9/20 - Loss: 0.0497
Epoch 10/20 - Loss: 0.0492
Epoch 11/20 - Loss: 0.0486
Epoch 12/20 - Loss: 0.0475
Epoch 13/20 - Loss: 0.0475
Epoch 14/20 - Loss: 0.0471
Epoch 15/20 - Loss: 0.0469
Epoch 16/20 - Loss: 0.0466
Epoch 17/20 - Loss: 0.0460
Epoch 18/20 - Loss: 0.0460
Epoch 19/20 - Loss: 0.0463
Epoch 20/20 - Loss: 0.0460
Final Test Accuracy: 98.28%


In [41]:
# Save the trained weights
model_path = "./Detection Model/"
torch.save(model.state_dict(), model_path + "analyst_model_mlp.pth")
print("Model weights saved successfully!")

# Save the scaler too: may be needed this to scale new data exactly like the training data
import joblib
joblib.dump(scaler, model_path + "scaler.pkl")

Model weights saved successfully!


['./Detection Model/scaler.pkl']

# Phase 3: Reinforcement Learning Environment

In [29]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import torch

class NetworkIntrusionEnv(gym.Env):
    """
    Custom Environment for Network Intrusion Response.
    
    The Agent (Responder) interacts with the environment by reading network traffic
    and the Analyst's prediction, then deciding to Allow, Block, or Throttle.
    """
    
    # ADDED max_steps here so the error goes away
    def __init__(self, X_data, y_labels, analyst_model=None, device='cpu', max_steps=2048):
        super(NetworkIntrusionEnv, self).__init__()
        
        # 1. Load Data & Model
        self.X_data = X_data  # Scaled Features (Numpy Array)
        self.y_labels = y_labels # Encoded Labels (Numpy Array)
        self.analyst_model = analyst_model
        self.device = device
        
        # Limit episode length to prevent "Endless Episode" syndrome
        self.max_steps = max_steps
        
        # Track position in the dataset
        self.current_step = 0
        self.dataset_size = len(X_data)
        self.steps_in_episode = 0 # New counter
        
        # 2. Define Action Space (The moves the agent can make)
        # 0 = Allow, 1 = Block, 2 = Throttle
        self.action_space = spaces.Discrete(3)
        
        # 3. Define Observation Space (The "eyes" of the agent)
        # Features + 1 extra for the Analyst's Confidence Score
        num_features = X_data.shape[1]
        self.observation_space = spaces.Box(
            low=0.0, 
            high=1.0, 
            shape=(num_features + 1,), 
            dtype=np.float32
        )

    def reset(self, seed=None, options=None):
        """
        Resets the environment to the start of a new episode.
        """
        super().reset(seed=seed)
        
        # Pick a random starting point to prevent memorizing the dataset order
        # Ensure we have enough room for a full episode
        self.current_step = np.random.randint(0, self.dataset_size - self.max_steps)
        self.steps_in_episode = 0  # Reset step counter
        
        # Get the first observation
        observation = self._get_observation()
        
        return observation, {}

    def step(self, action):
        """
        The Core Logic: Agent takes an action -> Environment returns Reward
        """
        # 1. Get Ground Truth for the current packet
        actual_label = self.y_labels[self.current_step]
        
        # Label 4 is "Normal" and everything else is "Attack"
        is_attack = (actual_label != 4) 
        
        # 2. Calculate Reward (The Teacher)
        reward = 0
        
        if action == 0: # ALLOW
            if not is_attack:
                reward = 1      # Correctly allowed normal traffic
            else:
                reward = -10    # FALSE NEGATIVE: Security Breach
                
        elif action == 1: # BLOCK
            if is_attack:
                reward = 10     # Correctly blocked an attack
            else:
                reward = -20    # FALSE POSITIVE: Disrupted normal user
                
        elif action == 2: # THROTTLE
            if is_attack:
                reward = 5      # Good, but blocking would have been better
            else:
                reward = -5     # Bad, annoyed a user, but didn't block them entirely
        
        # 3. Move to next packet
        self.current_step += 1
        self.steps_in_episode += 1
        
        # Check if episode is done (Limit reached OR End of Data reached)
        terminated = False
        if self.steps_in_episode >= self.max_steps or self.current_step >= self.dataset_size - 1:
            terminated = True
            
        # Get next observation
        observation = self._get_observation()
        
        return observation, reward, terminated, False, {}

    def _get_observation(self):
        """
        Helper to construct the state: [Traffic Features + Analyst Prediction]
        """
        # Get raw features
        features = self.X_data[self.current_step]
        
        # Get Analyst's opinion (if model is loaded)
        analyst_score = 0.5 # Default uncertainty
        NORMAL_CLASS_IDX = 4
        
        if self.analyst_model:
            with torch.no_grad():
                # Convert to tensor just for this one prediction
                obs_tensor = torch.tensor(features, dtype=torch.float32).unsqueeze(0).to(self.device)
                outputs = self.analyst_model(obs_tensor)
                
                # Get the probability that this is an ATTACK (sum of non-zero classes)
                # Assuming output is raw logits, apply Softmax
                probs = torch.softmax(outputs, dim=1)
                
                # Probability it is NOT normal (Class 4)
                prob_normal = probs[0][NORMAL_CLASS_IDX].item()
                analyst_score = 1.0 - prob_normal
        
        # Append Analyst Score to features
        full_state = np.append(features, analyst_score).astype(np.float32)
        return full_state

In [36]:
# 1. Re-create the empty model structure (The Skeleton)
input_shape = X_train_scaled.shape[1]
num_classes = len(np.unique(y_train)) 

# Re-initialize the class
model = AnalystModel(input_shape, num_classes)

# 2. Load the weights
model_path = "./Detection Model/"
state_dict = torch.load(model_path + "analyst_model_mlp.pth")
model.load_state_dict(state_dict)

# 3. Set to Evaluation Mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print("Model loaded successfully and ready for inference!")

# Now run your Environment Test
# Make sure to pass 'device' so the environment knows where to put the tensors
env = NetworkIntrusionEnv(
    X_train_scaled, 
    y_train.values, 
    analyst_model=model,
    device=device
)

# Reset and Test
obs, _ = env.reset()
print(f"Observation Shape: {obs.shape}")

action = env.action_space.sample()
obs, reward, done, _, _ = env.step(action)

print(f"Action Taken: {action}")
print(f"Reward Received: {reward}")
print("Environment is functional!")

Model loaded successfully and ready for inference!
Observation Shape: (79,)
Action Taken: 1
Reward Received: -20
Environment is functional!


In [31]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

# 1. Sanity Check
# SB3 has a built-in tool to check if your Gym environment follows all the rules.
# If this fails, it gives very helpful error messages.
print("Checking environment compatibility...")
check_env(env)
print("Environment is compliant!")

# 2. Initialize the PPO Agent
# verbose=1 shows a progress log (rewards, loss, etc.)
model_rl = PPO(
    "MlpPolicy", 
    env, 
    verbose=1, 
    learning_rate=0.0003,
    gamma=0.99  # Discount factor (values immediate rewards vs long-term)
)

# 3. Train the Agent
print("Starting training (this may take a few minutes)...")
# 100,000 steps is a good starting point for a prototype.
model_rl.learn(total_timesteps=100000)

print("Training complete!")

Checking environment compatibility...
Environment is compliant!
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Starting training (this may take a few minutes)...
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 2.05e+03  |
|    ep_rew_mean     | -1.14e+04 |
| time/              |           |
|    fps             | 298       |
|    iterations      | 1         |
|    time_elapsed    | 6         |
|    total_timesteps | 2048      |
----------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.05e+03    |
|    ep_rew_mean          | -1.05e+04   |
| time/                   |             |
|    fps                  | 248         |
|    iterations           | 2           |
|    time_elapsed         | 16          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl       

In [43]:
import numpy as np

def evaluate_agent(env, model, num_steps=1000):
    obs, _ = env.reset()
    
    # Initialize metrics to avoid UnboundLocalError
    total_attacks = 0
    attacks_blocked = 0
    attacks_missed = 0
    
    total_normal = 0
    normal_allowed = 0
    normal_blocked = 0
    
    # Initialize rates to 0.0 in case of empty counters
    detection_rate = 0.0
    fp_rate = 0.0
    
    print(f"Starting evaluation on {num_steps} packets...")
    
    for _ in range(num_steps):
        action, _ = model.predict(obs, deterministic=True)
        
        current_idx = env.current_step
        actual_label = env.y_labels[current_idx]
        is_attack = (actual_label != 4)
        
        obs, reward, done, _, _ = env.step(action)
        
        if is_attack:
            total_attacks += 1
            if action == 1: 
                attacks_blocked += 1
            elif action == 2:
                attacks_blocked += 0.5 
            else:
                attacks_missed += 1
        else:
            total_normal += 1
            if action == 0:
                normal_allowed += 1
            elif action == 1:
                normal_blocked += 1
            
        if done:
            obs, _ = env.reset()

    # --- AGENT REPORT CARD ---
    print("\n--- AGENT REPORT CARD ---")
    
    if total_attacks > 0:
        detection_rate = (attacks_blocked / total_attacks) * 100
        print(f"Attack Mitigation Rate: {detection_rate:.2f}%")
    else:
        print("Attack Mitigation Rate: N/A (No attacks in sample)")
        
    if total_normal > 0:
        fp_rate = (normal_blocked / total_normal) * 100
        print(f"False Positive Rate:    {fp_rate:.2f}% (Lower is better)")
    else:
        print("False Positive Rate:    N/A (No normal traffic in sample)")
    
    accuracy = (attacks_blocked + normal_allowed) / num_steps * 100
    print(f"Overall Response Accuracy: {accuracy:.2f}%")
    
    return detection_rate, fp_rate

evaluate_agent(env, model_rl, num_steps=5000)

Starting evaluation on 5000 packets...

--- AGENT REPORT CARD ---
Attack Mitigation Rate: 96.79%
False Positive Rate:    1.36% (Lower is better)
Overall Response Accuracy: 98.18%


(96.78714859437751, 1.3581890812250332)

In [33]:
from sklearn.utils import shuffle

# 1. Merge X and y back together temporarily to shuffle them in sync
# (This ensures the labels stick to the correct rows)
X_train_shuffled, y_train_shuffled = shuffle(X_train_scaled, y_train.values, random_state=42)

print("Data shuffled! Now the agent will see a mix of traffic.")

# 2. Re-initialize the Environment with the SHUFFLED data
env = NetworkIntrusionEnv(
    X_train_shuffled, 
    y_train_shuffled, 
    analyst_model=model, 
    device=device,
    max_steps=2048 # Keep the mini-episodes!
)

# 3. Re-train the PPO Agent
# (We need to start fresh so it un-learns the "Block Everything" habit)
model_rl = PPO("MlpPolicy", env, verbose=1, learning_rate=0.0003)
model_rl.learn(total_timesteps=100000)

print("Retraining complete.")

Data shuffled! Now the agent will see a mix of traffic.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 2.05e+03  |
|    ep_rew_mean     | -1.16e+04 |
| time/              |           |
|    fps             | 326       |
|    iterations      | 1         |
|    time_elapsed    | 6         |
|    total_timesteps | 2048      |
----------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.05e+03    |
|    ep_rew_mean          | -1.04e+04   |
| time/                   |             |
|    fps                  | 308         |
|    iterations           | 2           |
|    time_elapsed         | 13          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011319814 |
|    clip_fraction        | 0.163     

In [44]:
evaluate_agent(env, model_rl, num_steps=5000)

Starting evaluation on 5000 packets...

--- AGENT REPORT CARD ---
Attack Mitigation Rate: 98.06%
False Positive Rate:    1.09% (Lower is better)
Overall Response Accuracy: 98.70%


(98.05982215036379, 1.089556205155461)

In [42]:
# 1. Save the RL Agent (The Responder)
agent_path = "./Responder Agent/"
model_rl.save(agent_path + "ppo_response_agent")